Task 2: Exploratory Data Analysis (EDA)
Overview of the Data:

Understand the structure of the dataset, including the number of rows, columns, and data types.
Summary Statistics:

Understand the central tendency, dispersion, and shape of the dataset’s distribution.
Distribution of Numerical Features:

Visualize the distribution of numerical features to identify patterns, skewness, and potential outliers.
Distribution of Categorical Features:

Analyze the distribution of categorical features to gain insights into the frequency and variability of categories.
Correlation Analysis:

Understand the relationship between numerical features.
Identifying Missing Values:

Identify missing values to determine missing data and decide on appropriate imputation strategies.
Outlier Detection:

Use box plots to identify outliers.

In [1]:
import os , sys
sys.path.append(os.path.join(os.path.abspath('..')))
# Import modules
from src import data_loader as dl
from scripts.Eda import data_overview, summary_statistics,identify_missing_values, distribution_of_numerical_features, correlation_analysis,distribution_of_categorical_features

2025-03-07 16:32:25,666 - INFO - Directory already exists: c:\Users\HP\Documents\Dagii\Credit-scoring-model\data
2025-03-07 16:32:25,668 - INFO - Directory already exists: c:\Users\HP\Documents\Dagii\Credit-scoring-model\src\../logs


In [2]:
#import data
data = dl.load_data("data.csv")

2025-03-07 16:32:40,090 - INFO - Attempting to load data from: c:\Users\HP\Documents\Dagii\Credit-scoring-model\data\data.csv
2025-03-07 16:32:40,661 - INFO - Successfully loaded data from: c:\Users\HP\Documents\Dagii\Credit-scoring-model\data\data.csv


In [3]:
data_overview(data)

Number of rows and columns: (95662, 16)
Column names and data types:
 TransactionId            object
BatchId                  object
AccountId                object
SubscriptionId           object
CustomerId               object
CurrencyCode             object
CountryCode               int64
ProviderId               object
ProductId                object
ProductCategory          object
ChannelId                object
Amount                  float64
Value                     int64
TransactionStartTime     object
PricingStrategy           int64
FraudResult               int64
dtype: object


2. Summary Statistics